<a href="https://colab.research.google.com/github/BitKnitting/FitHome_Analysis/blob/master/notebooks/Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calculating Baseline
The active part of the FitHome experience begins with knowledge of initial consumption.

#Data from Firebase
Retrieve the readings into a json file.

In [1]:
%%time
!curl 'https://fithome-9ebbd.firebaseio.com/flower-09282019/readings.json?print=pretty' > baseline.json 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18.0M  100 18.0M    0     0  2065k      0  0:00:08  0:00:08 --:--:-- 5406k
CPU times: user 80.7 ms, sys: 20.8 ms, total: 102 ms
Wall time: 10.4 s


In [2]:
# @title Put into a Pandas dataframe
# This can take >= 3 minutes.
%%time
import pandas as pd
df_needs_reshape = pd.read_json("baseline.json","r", encoding="utf8")
df_reshaped = df_needs_reshape.T
df_reshaped.head()

CPU times: user 4min 4s, sys: 7.14 s, total: 4min 11s
Wall time: 4min


In [0]:
# The datetime is in UTC time.  We are in Kirkland, WA
df_reshaped.index = df_reshaped.index.tz_localize('UTC').tz_convert('US/Pacific')

In [4]:
# @title Start Date and End Data of Series
print('Start date: {}'.format(df_reshaped.index.min()))
print('End   date: {}'.format(df_reshaped.index.max()))
time_between = df_reshaped.index.max() - df_reshaped.index.min()
print('\n\nElapsed time: {}'.format(time_between))

Start date: 2019-10-11 13:46:35-07:00
End   date: 2019-10-31 08:40:27-07:00


Elapsed time: 19 days 18:53:52


# Describe the Data
Let's look at some interesting stats

In [5]:
# @title Interesting Data Stats
df_reshaped.describe()

,I,P
count,314616.000000,314616.000000
mean,6.175310,660.158902
std,4.876879,546.450344
min,0.002000,0.000320
25%,2.525000,259.579800
50%,4.309000,479.150850
75%,8.495000,896.704275
max,52.895000,5663.446000


# Daily Average Electricity Use

- goup data into days.
- use only days which have enough samples in each hour.

In [19]:
all_power_data.resample(rule='H').count() > 600

2019-10-11 13:00:00-07:00    False
2019-10-11 14:00:00-07:00     True
2019-10-11 15:00:00-07:00     True
2019-10-11 16:00:00-07:00     True
2019-10-11 17:00:00-07:00     True
                             ...  
2019-10-31 04:00:00-07:00     True
2019-10-31 05:00:00-07:00     True
2019-10-31 06:00:00-07:00     True
2019-10-31 07:00:00-07:00     True
2019-10-31 08:00:00-07:00    False
Freq: H, Name: P, Length: 476, dtype: bool

In [25]:
count_per_day = all_power_data.resample(rule='D').count() > 17350
count_per_day

2019-10-11 00:00:00-07:00    False
2019-10-12 00:00:00-07:00    False
2019-10-13 00:00:00-07:00    False
2019-10-14 00:00:00-07:00    False
2019-10-15 00:00:00-07:00    False
2019-10-16 00:00:00-07:00    False
2019-10-17 00:00:00-07:00    False
2019-10-18 00:00:00-07:00    False
2019-10-19 00:00:00-07:00    False
2019-10-20 00:00:00-07:00    False
2019-10-21 00:00:00-07:00    False
2019-10-22 00:00:00-07:00     True
2019-10-23 00:00:00-07:00     True
2019-10-24 00:00:00-07:00     True
2019-10-25 00:00:00-07:00     True
2019-10-26 00:00:00-07:00    False
2019-10-27 00:00:00-07:00     True
2019-10-28 00:00:00-07:00     True
2019-10-29 00:00:00-07:00    False
2019-10-30 00:00:00-07:00    False
2019-10-31 00:00:00-07:00    False
Freq: D, Name: P, dtype: bool

In [13]:
all_power_data = df_reshaped['P']
# Convert readings to kWh
p_hourly_kWh = all_power_data.resample(rule='H').mean()/1000

display(p_hourly_kWh.describe())

count    446.000000
mean       0.664325
std        0.426886
min        0.157123
25%        0.314058
50%        0.539219
75%        0.953748
max        2.220034
Name: P, dtype: float64

In [15]:
# Set criteria for hourly data to include at least 600 samples.
p_hourly_kWh = p_hourly_kWh.resample(rule='H').count() > 600
p_hourly_kWh.describe()


count       476
unique        1
top       False
freq        476
Name: P, dtype: object